In [ ]:
!pip -q install openai langchain==0.0.99rc0

## Basics with OpenAI API

In [ ]:
import os
import openai

openai.api_key =''
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello what kind of assistant are you?"},
    ]
)

In [ ]:
response

<OpenAIObject chat.completion id=chatcmpl-6pakUtAnnDzKpBkh2QcxLAp1ymTtR at 0x7fdae98f9400> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I am a virtual assistant, equipped with AI technology to assist you with various tasks and answer your questions as best as I can. How may I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1677754010,
  "id": "chatcmpl-6pakUtAnnDzKpBkh2QcxLAp1ymTtR",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 35,
    "prompt_tokens": 26,
    "total_tokens": 61
  }
}

### Chat Markup Language - token system

```markdown
<|im_start|>system
You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible.
Knowledge cutoff: 2021-09-01
Current date: 2023-03-01<|im_end|>
<|im_start|>user
How are you<|im_end|>
<|im_start|>assistant
I am doing well!<|im_end|>
<|im_start|>user
How are you now?<|im_end|>
```

```
import openai

openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
```

In [ ]:
messages=[
        {"role": "system", "content": "You are a helpful assistant named Kate."},
        {"role": "user", "content": "Hello what kind of assistant are you?"},
    ]

In [ ]:
conversation_total_tokens = 0

while True:
    message = input("Human: ")
    if message=='exit':
        print(f"{conversation_total_tokens} tokens used in total in this conversation")
        break
    if message:
        messages.append(
            {"role": "user", "content": message},
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
    
    reply = response.choices[0].message.content
    total_tokens = response.usage['total_tokens']
    conversation_total_tokens += total_tokens
    print(f"ChatGPT: {reply} \n {total_tokens} tokens used")
    messages.append({"role": "assistant", "content": reply})

Human: When was Marcus Aurelius emperor of Rome?
ChatGPT: Hello! I'm Kate, a virtual assistant here to help you. The Roman emperor Marcus Aurelius ruled from 161 AD to his death in 180 AD. He was the last of the Five Good Emperors of Rome. 
 91 tokens used
Human: Who was his wife?
ChatGPT: Marcus Aurelius' wife was named Faustina the Younger. She was also his first cousin and they were married in 145 AD. They had 13 children together, many of whom did not survive childhood. Faustina was known for her intelligence, beauty, and devotion to her husband. She was later deified after her death. 
 176 tokens used
Human: how many children did they have?
ChatGPT: Marcus Aurelius and Faustina the Younger had 14 children together, including 9 daughters and 5 sons. However, most of their children died at an early age, and only a few survived into adulthood. Their most famous surviving child was Annia Galeria Faustina, who became the wife of Marcus Aurelius' co-emperor and adopted brother, Lucius V

## ChatGPT with LangChain

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.99rc0
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.8/dist-packages
Requires: aiohttp, aleph-alpha-client, dataclasses-json, deeplake, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI, OpenAIChat

In [ ]:
prefix_messages = [{"role": "system", "content": "You are a helpful history professor named Kate."}]


In [ ]:
## old way
# llm = OpenAI(model_name="text-davinci-003",
#              temperature=0, )

## New way
llm = OpenAIChat(model_name='gpt-3.5-turbo', 
             temperature=0, 
             prefix_messages=prefix_messages,
             max_tokens = 256)

In [ ]:

template = """Take the following question: {user_input}

Answer it in an informative and intersting but conscise way for someone who is new to this topic."""

prompt = PromptTemplate(template=template, 
                        input_variables=["user_input"])


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

user_input = "When was Marcus Aurelius the emperor of Rome?"

llm_chain.run(user_input)

'Marcus Aurelius was the emperor of Rome from 161 to 180 AD. He was known for his philosophical writings, particularly his book "Meditations," which is still studied today. During his reign, he faced challenges such as wars with Germanic tribes and a devastating plague. Despite these difficulties, he is remembered as one of Rome\'s "Five Good Emperors" for his efforts to improve the lives of his subjects and his commitment to justice and virtue.'

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

user_input = "Who was Marcus Aurelius married to?"

llm_chain.run(user_input)

'Marcus Aurelius was married to a woman named Faustina the Younger. She was the daughter of Antoninus Pius, who was the emperor before Marcus Aurelius. Faustina was known for her beauty and intelligence, and she was a devoted wife to Marcus Aurelius. However, there were rumors that she was unfaithful to him, which caused him great distress. Despite this, Marcus Aurelius remained loyal to her and even deified her after her death.'